---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    graph_types = []
    
    for G in P1_Graphs:
        
        average_clustering = nx.average_clustering(G)
        
        average_shortest_path_length = nx.average_shortest_path_length(G)
        
        degrees = G.degree()
        degree_values_sorted = sorted(set(degrees.values()))
        degree = [list( degrees.values() ).count(i) / float(nx.number_of_nodes(G) ) for i in degree_values_sorted]

        if len(degree)>10:
            graph_types.append('PA')
        elif average_clustering < 0.1:
            graph_types.append('SW_H')
        else:
            graph_types.append('SW_L')
    
    return graph_types


graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

def salary_predictions():
    # Create DataFrame
    df = pd.DataFrame(index=G.nodes())
    graph_data = G.nodes(data=True)
    df['Department'] = pd.Series([node[1]['Department'] for node in graph_data])
    df['ManagementSalary'] = pd.Series([node[1]['ManagementSalary'] for node in graph_data])
    
    #Add features
    df['Degree'] = list(G.degree().values())
    df['Clustering'] = list(nx.clustering(G).values())
    df['Degree centrality'] = nx.degree_centrality(G).values()
    df['Closeness centrality'] = nx.closeness_centrality(G).values()
    df['Betweenness centrality'] = nx.betweenness_centrality(G).values()
    df['Page Rank'] = nx.pagerank(G).values()
    
    # Train-test split
    df_train = df[~pd.isnull(df['ManagementSalary'])]
    df_test = df[pd.isnull(df['ManagementSalary'])]
    y_train, X_train = df_train.pop('ManagementSalary'), df_train
    _, X_test = df_test.pop('ManagementSalary'), df_test
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    params = {'C': [0.1, 1.0]}
    model = GridSearchCV(LogisticRegression(), cv=5, param_grid=params, scoring='roc_auc').fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_test)
    
    return pd.Series(y_pred[:, 1])


salary_predictions()

0      0.055790
1      0.771315
2      0.999994
3      0.115621
4      0.191691
5      0.270835
6      0.272524
7      0.274797
8      0.087888
9      0.093234
10     0.038064
11     0.078031
12     0.151121
13     0.194803
14     0.466986
15     0.054430
16     0.999849
17     0.954391
18     0.059555
19     0.100698
20     0.072681
21     0.030442
22     0.172256
23     0.099261
24     0.416577
25     0.010790
26     0.489622
27     0.833889
28     0.093534
29     0.616086
         ...   
222    0.017623
223    0.043051
224    0.000051
225    0.109880
226    0.006979
227    0.074325
228    0.031736
229    0.004135
230    0.001517
231    0.000087
232    0.013116
233    0.160812
234    0.103202
235    0.010279
236    0.008572
237    0.000148
238    0.000068
239    0.124481
240    0.080679
241    0.087163
242    0.095701
243    0.003357
244    0.116472
245    0.083100
246    0.123396
247    0.000298
248    0.000215
249    0.000088
250    0.030915
251    0.082927
Length: 252, dtype: floa

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [51]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [55]:
def new_connections_predictions():
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    
    # DataFrame with features
    df = pd.DataFrame(index=list(nx.non_edges(G)))
    df['Jaccard coefficient']  = [score for u, v, score in nx.jaccard_coefficient(G)]
    df['Preferential attachment']  = [score for u, v, score in nx.preferential_attachment(G)]
    df['Resource allocation']  = [score for u, v, score in nx.resource_allocation_index(G)]
    
    # Community structure features
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    df['CN Doundarajan Hopcroft']  = [score for u, v, score in nx.cn_soundarajan_hopcroft(G)]

    # Join with future_connections DataFrame
    future_connections = future_connections.join(df, how='outer')
    
    # Train-test split
    future_connections_train = future_connections[~pd.isnull(future_connections['Future Connection'])]
    future_connections_test = future_connections[pd.isnull(future_connections['Future Connection'])]
    
    y_train, X_train = future_connections_train.pop('Future Connection'), future_connections_train
    _, X_test = future_connections_test.pop('Future Connection'), future_connections_test
    
    # Scale the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Fit the model and predict
    params = {'C': [0.1, 1.0]}
    model = GridSearchCV(LogisticRegression(), cv=5, param_grid=params, scoring='roc_auc').fit(X_train, y_train)
    print(model.best_score_)
    
    y_pred = model.predict_proba(X_test)
    
    return pd.Series(y_pred[:, 1])


new_connections_predictions()

0.911921779805


0         0.027418
1         0.062277
2         0.278451
3         0.011869
4         0.007395
5         0.808372
6         0.019005
7         0.009802
8         0.547941
9         0.043279
10        0.316971
11        0.035679
12        0.020191
13        0.010665
14        0.158433
15        0.029695
16        0.015717
17        0.135426
18        0.027047
19        0.953117
20        0.013915
21        0.010969
22        0.010369
23        0.008514
24        0.030754
25        0.039503
26        0.748736
27        0.081719
28        0.179193
29        0.007465
            ...   
122082    0.011475
122083    0.011490
122084    0.067318
122085    0.011465
122086    0.011490
122087    0.011485
122088    0.011485
122089    0.011485
122090    0.011485
122091    0.011446
122092    0.025144
122093    0.011485
122094    0.011485
122095    0.011485
122096    0.011487
122097    0.011487
122098    0.011473
122099    0.011451
122100    0.011480
122101    0.011492
122102    0.011492
122103    0.